In [1]:
# requires torch, torch_geometric, open3d, plotly
# open3d needs python 3.10, anything higher will not work

import torch
from torch_geometric.datasets import ModelNet
from torch_geometric.transforms import SamplePoints, NormalizeScale
from torch_geometric.loader import DataLoader

import open3d as o3d
import plotly.graph_objects as go

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import datetime
import os
import random

import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# pointtransformer fully implemented, consolidated to another file for import
from pointtransformer import PointTransformerClassifier

In [3]:
# change to your device, also change device in pointtransformer.py file
device = "cuda"

In [ ]:
# modelnet10 dataset config

num_points = 1024

pre_transform = NormalizeScale()
transform = SamplePoints(num_points)

batch_size = 64

root = 'data/ModelNet10'
dataset_train = ModelNet(root=root, name='10', train=True, pre_transform=pre_transform, transform=transform)
trainloader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)

dataset_test = ModelNet(root=root, name='10', train=False, pre_transform=pre_transform, transform=transform)
testloader = DataLoader(dataset_test, batch_size=batch_size)

print(f'Number of training examples: {len(dataset_train)}')
print(f'Number of test examples: {len(dataset_test)}')

classes = dataset_test.raw_file_names
print(classes)

In [4]:
# modelnet40 dataset config

num_points = 1024

pre_transform = NormalizeScale()
transform = SamplePoints(num_points)

batch_size = 64

root = 'data/ModelNet40'
dataset_train = ModelNet(root=root, name='40', train=True, pre_transform=pre_transform, transform=transform)
trainloader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)

dataset_test = ModelNet(root=root, name='40', train=False, pre_transform=pre_transform, transform=transform)
testloader = DataLoader(dataset_test, batch_size=batch_size)

classes = ["airplane", "bathtub", "bed", "bench", "bookshelf", "bottle", "bowl", "car", "chair", "cone", "cup", "curtain", "desk", "door", "dresser", "flower_pot", "glass_box", "guitar", "keyboard", "lamp", "laptop", "mantel", "monitor", "night_stand", "person", "piano", "plant", "radio", "range_hood", "sink", "sofa", "stairs", "stool", "table", "tent", "toilet", "tv_stand", "vase", "wardrobe", "xbox"]

print(f'Number of training examples: {len(dataset_train)}')
print(f'Number of test examples: {len(dataset_test)}')
print(classes)

Number of training examples: 9843
Number of test examples: 2468
['airplane', 'bathtub', 'bed', 'bench', 'bookshelf', 'bottle', 'bowl', 'car', 'chair', 'cone', 'cup', 'curtain', 'desk', 'door', 'dresser', 'flower_pot', 'glass_box', 'guitar', 'keyboard', 'lamp', 'laptop', 'mantel', 'monitor', 'night_stand', 'person', 'piano', 'plant', 'radio', 'range_hood', 'sink', 'sofa', 'stairs', 'stool', 'table', 'tent', 'toilet', 'tv_stand', 'vase', 'wardrobe', 'xbox']


In [5]:
# sanity check, plot the first element of training data

data = dataset_test[0]

fig = go.Figure(
  data=[
    go.Scatter3d(
      x=data.pos[:,0], y=data.pos[:,1], z=data.pos[:,2],
      mode='markers',
      marker=dict(size=1, color="white"))],
  layout=dict(
    scene=dict(
      xaxis=dict(visible=False),
      yaxis=dict(visible=False),
      zaxis=dict(visible=False))))

fig.update_layout(template='plotly_dark')

fig.show()

In [6]:
# create a new pointtransformer
pointtransformer = PointTransformerClassifier(num_classes=40)
pointtransformer.to(device)

PointTransformerClassifier(
  (backbone): PointTransformerBackbone(
    (init_linear): Linear(in_features=3, out_features=32, bias=True)
    (transformer1): PointTransformer(
      (fcl1): Linear(in_features=32, out_features=32, bias=True)
      (head1): TransformerLayer(
        (query): Linear(in_features=32, out_features=32, bias=True)
        (key): Linear(in_features=32, out_features=32, bias=True)
        (value): Linear(in_features=32, out_features=32, bias=True)
        (pos_enc): Sequential(
          (0): Conv1d(3, 32, kernel_size=(1,), stride=(1,))
          (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU()
          (3): Conv1d(32, 32, kernel_size=(1,), stride=(1,))
          (4): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (5): ReLU()
        )
      )
      (head2): TransformerLayer(
        (query): Linear(in_features=32, out_features=32, bias=True)
        (key): Linea

In [7]:
# quick sanity check
test_data = torch.rand(batch_size, num_points, 3).to(device)

output = pointtransformer(test_data)
print(output.shape)

torch.Size([64, 40])


In [ ]:
# training hyperparameters
num_epochs = 50
learning_rate = 0.01
momentum = 0.7
weight_decay = 0

optimizer = optim.SGD(pointtransformer.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss().to(device)

In [18]:
# load a pointtransformer from a saved state
checkpoint_pointnet = torch.load("pointtransformer_modelnet40/187.pth", map_location=torch.device(device))
pointtransformer.load_state_dict(checkpoint_pointnet['model_state_dict'])

<All keys matched successfully>

In [ ]:
# training loop

directory = "./pointtransformer_modelnet40_final_again"
os.makedirs(directory, exist_ok=True)

for i in range(4):

    for epoch in range(num_epochs):

        train_accuracy = 0
        test_accuracy = 0
        loss_avg = 0
        count = 0

        pointtransformer.train()
        for data in trainloader:

            clouds = data.pos.view(data.batch[-1]+1, num_points, 3).to(device)
            labels = data.y.to(device)

            optimizer.zero_grad()

            outputs = pointtransformer(clouds)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            loss_avg += loss.item()
            count += 1
        
        loss_avg = loss_avg/count
        
        pointtransformer.eval()
        with torch.no_grad():
            correct = 0
            total = 0
            for data in testloader:
                
                clouds = data.pos.view(data.batch[-1]+1, num_points, 3).to(device)
                labels = data.y.to(device)
                
                outputs = pointtransformer(clouds)

                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

            test_accuracy = correct/total
            correct = 0
            total = 0
        
        #print("{}   [Epoch {:3}]  Loss: {:8.4}  Accuracy:   {:8.4}%".format(datetime.datetime.now(), epoch, loss_avg, 100*accuracy))
        #print("{:8.4},{:8.4},{:8.4}".format(loss_avg, 100*train_accuracy, 100*test_accuracy))
        print("{:8.4},{:8.4}".format(loss_avg, 100*test_accuracy))

        torch.save(
            {'model_state_dict': pointtransformer.state_dict()},
            directory + "/{:03d}".format(epoch+i*num_epochs) + ".pth")
    
    learning_rate *= 0.1
    optimizer = optim.SGD(pointtransformer.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)

In [20]:
# test evaluation

pointtransformer.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for data in testloader:
        
        clouds = data.pos.view(data.batch[-1]+1, num_points, 3).to(device)
        labels = data.y.to(device)
            
        outputs = pointtransformer(clouds)

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    accuracy = correct/total

    print("{}   Accuracy:   {:8.4}%".format(datetime.datetime.now(), 100*accuracy))

2025-03-21 14:52:46.382171   Accuracy:      86.79%


In [ ]:
# test on random point clouds

idx = random.randint(0, len(dataset_test))
data = dataset_test[idx]
cloud = data.pos.view(1, num_points, 3).to(device)

output = pointtransformer(cloud)
probabilities = 100*F.softmax(output.transpose(1,0), dim=0)

_, predicted = torch.max(output.data, 1)
label = data.y

print('Predicted Class: {}    Certainty: {:8.4}   Actual Class:   {}'.format(classes[predicted.item()], probabilities[predicted.item()].item(), classes[label.item()]))

fig = go.Figure(
  data=[
    go.Scatter3d(
      x=data.pos[:,0], y=data.pos[:,1], z=data.pos[:,2],
      mode='markers',
      marker=dict(size=1, color="white"))],
  layout=dict(
    scene=dict(
      xaxis=dict(visible=False),
      yaxis=dict(visible=False),
      zaxis=dict(visible=False))))

fig.update_layout(template='plotly_dark')

fig.show()

In [ ]:
sum(p.numel() for p in pointtransformer.parameters())

In [ ]:
pred = []
true = []

for data in testloader:
    
    clouds = data.pos.view(data.batch[-1]+1, num_points, 3).to(device)
    labels = data.y.to(device)

    output = pointtransformer(clouds)

    output = (torch.max(torch.exp(output), 1)[1]).data.cpu().numpy()
    pred.extend(output) # Save Prediction
    
    labels = labels.data.cpu().numpy()
    true.extend(labels) # Save Truth

cf_matrix = confusion_matrix(true, pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cf_matrix,
                              display_labels=classes)
disp.plot(cmap=plt.cm.Blues, xticks_rotation="vertical")

plt.savefig("pointtransformer_modelnet10_confusion_matrix.svg", format="svg")

In [ ]:
for num_points in [256, 512, 1024, 2048, 4096]:

    pre_transform = NormalizeScale()
    transform = SamplePoints(num_points)

    batch_size = 64

    root = 'data/ModelNet40'
    dataset_train = ModelNet(root=root, name='40', train=True, pre_transform=pre_transform, transform=transform)
    trainloader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)

    dataset_test = ModelNet(root=root, name='40', train=False, pre_transform=pre_transform, transform=transform)
    testloader = DataLoader(dataset_test, batch_size=batch_size)

    classes = dataset_test.raw_file_names
    
    accuracy = 0
    total = 0
    correct = 0

    pointtransformer.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for data in testloader:
            
            clouds = data.pos.view(data.batch[-1]+1, num_points, 3).to(device)
            labels = data.y.to(device)
                
            outputs = pointtransformer(clouds)

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        accuracy = correct/total

        print("{:8.4}%".format(100*accuracy))

  0.1101%
  0.4405%
  0.3304%
     0.0%


KeyboardInterrupt: 